# TRAVELING SALESMAN: SIMULATED ANNEALING
Simulated annealing (SA) is a probabilistic technique for approximating the global optimum of a given function. Specifically, it is a metaheuristic to approximate global optimization in a large search space. It is often used when the search space is discrete (e.g., all tours that visit a given set of cities).

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
from math import floor
from math import inf
from math import exp, expm1
import random
from random import randint
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
import scipy 
from scipy.misc import comb # comb(n,k, exact=True)
import decimal

# from scipy import special
# from scipy.special import comb
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [2]:
dist_mat = 'distance_matrix'
def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns

In [3]:
df_distMat = read_data('%s.csv'%dist_mat)
print("rows(%s) x cols(%s) "%check_packaging(df_distMat))
print()
print("%s"%data_check(df_distMat))
print()
print(check_ns(df_distMat))
print()
num_cities = df_distMat.shape[0]
# df_cityDist.set_index('city')
print('Number of cities including depot: ', num_cities)

rows(6) x cols(6) 

      0     1     2     3     4     5
0   NaN  36.0  32.0  54.0  20.0  40.0
1  36.0   NaN  22.0  58.0  54.0  67.0
2  32.0  22.0   NaN  36.0  42.0  71.0

              0          1          2          3          4         5
count   5.00000   5.000000   5.000000   5.000000   5.000000   5.00000
mean   36.40000  47.400000  40.600000  58.000000  42.200000  63.00000
std    12.36123  18.132843  18.487834  20.736441  13.236314  21.05944
min    20.00000  22.000000  22.000000  36.000000  20.000000  40.00000
25%    32.00000  36.000000  32.000000  50.000000  42.000000  45.00000
50%    36.00000  54.000000  36.000000  54.000000  45.000000  67.00000
75%    40.00000  58.000000  42.000000  58.000000  50.000000  71.00000
max    54.00000  67.000000  71.000000  92.000000  54.000000  92.00000

Number of cities including depot:  6


In [4]:
df_distMat.fillna(inf,inplace = True)
df_distMat
# df_distMat.iloc[1,0]

,0,1,2,3,4,5
0,inf,36.000000,32.000000,54.000000,20.000000,40.000000
1,36.000000,inf,22.000000,58.000000,54.000000,67.000000
2,32.000000,22.000000,inf,36.000000,42.000000,71.000000
3,54.000000,58.000000,36.000000,inf,50.000000,92.000000
4,20.000000,54.000000,42.000000,50.000000,inf,45.000000
5,40.000000,67.000000,71.000000,92.000000,45.000000,inf


Note: The code below has merely reporoduced been reproduced in Python, programming language, with slight adaptations from Mr Woolway's SA code.

#NOTE: MATLAB indexes from 1

## SIMPLE SA CODE:

In [5]:
#Helper function. Simply computes a distance of a proposed path
 
# INPUTS: 
#          distMatrix   - the distance matrix: pandas dataframe structure
#          proposedPath - the sample path to compute : numpy array
#
# OUTPUTS: 
#          totaldist    - the corresponding distance of the proposed path

In [6]:
#The distance calculator returns the sum of the distances between states and adds the distance from the last state to the first(depot)
# def distCalculatorReturn1(distMatrix, proposedPath):
#     dist = 0
#     i = 0
#     len_proposedPath = proposedPath.shape[0]
#     while i != len_proposedPath-1:
#         dist = dist + distMatrix.iloc[proposedPath[i], proposedPath[i+1]]
#         i = i + 1
#     totaldist = dist +distMatrix.iloc[proposedPath[-1], proposedPath[0]]
#     return totaldist


def distCalculatorReturn1(distMatrix, proposedPath):
    dist = 0
    i = 0
    len_proposedPath = proposedPath.shape[0]
    for i in range(len_proposedPath-1):
        dist = dist + distMatrix.iloc[proposedPath[i], proposedPath[i+1]]
    totaldist = dist +distMatrix.iloc[proposedPath[-1], proposedPath[0]]
    return totaldist

In [7]:
#INPUTS: 
#D    - some given distance matrix (symmetric and 0's on diag)
#n    - the number of cities to vist

#OUTPUTS:
#path - the optimal path according to the simulated annealing
#dist - the corresponding distance to the optimal path
#distCal - the distance of the path calculated by the distance matrix
def simulates_annealing1(D): 
    number_states = D.shape[0]
    n = number_states
    T = 100000 #Initial temperature
    L = 20*n #Length of the markov chain
    e = (1*exp(1))**(-9) #Tolerance
    
    
    x  = np.random.permutation(range(0,number_states)) # creates an array of all the states including the depot in a random order
    fx = distCalculatorReturn1(D, x) #calculates the distance of the random path
    
    while T > e:
        for i in range(0,L):
            rand1 = decimal.Decimal(random.randrange(100))/100
            rand2 = decimal.Decimal(random.randrange(100))/100
            num1 = int(rand1*n)
            num2 = int(rand2*n)
            
            #ensuring num1 and num2 aren't equal
            while num1==num2:
                rand3 = decimal.Decimal(random.randrange(100))/100
                num1 = int(rand3*n)
                
            y = x # copy of the random path and making random changes to compare two paths
            #make some changes to y_path
            swap1 = y[num1]
            y[num1] = y[num2]
            y[num2] = swap1
            fy = distCalculatorReturn1(D,y)#calculate the distance of the y_path i.e the changes x_path
            
            if fy < fx:
                x = y
                fx = fy
            elif decimal.Decimal(random.randrange(100))/100 < exp(1)**(-(fy-fx)/T):
                x = y
                fx = fy
            
        T=0.9*T
    path = x
    dist = fx 
    dist_calc_path = distCalculatorReturn1(D,x)
    path_y = y
    dist_y = fy 
    dist_calc_path_y = distCalculatorReturn1(D,y)

    return path, dist, dist_calc_path #path_y, dist_y, dist_calc_path_y

In [8]:
path, dist, distCalc = simulates_annealing1(df_distMat)
display(path)
display(dist)
display(distCalc)
#, path_y, dist_y, dist_calc_path_y
# display(path_y)
# display(dist_y)
# display(dist_calc_path_y)


array([1, 0, 4, 5, 2, 3])

229.0

266.0

In [9]:
display(df_distMat)

,0,1,2,3,4,5
0,inf,36.000000,32.000000,54.000000,20.000000,40.000000
1,36.000000,inf,22.000000,58.000000,54.000000,67.000000
2,32.000000,22.000000,inf,36.000000,42.000000,71.000000
3,54.000000,58.000000,36.000000,inf,50.000000,92.000000
4,20.000000,54.000000,42.000000,50.000000,inf,45.000000
5,40.000000,67.000000,71.000000,92.000000,45.000000,inf


## SIMULATED ANNEALING
### Specific to the problem
#### The first and last state remain fixed (the depot)


In [10]:
#Helper function. Simply computes a distance of a proposed path
 
# INPUTS: 
#          distMatrix   - the distance matrix: pandas dataframe structure
#          proposedPath - the sample path to compute : numpy array
#
# OUTPUTS: 
#          totaldist    - the corresponding distance of the proposed path

In [11]:
#The distance calculator returns the sum of the distances between states; starting and ending at the depot
# def distCalculator(distMatrix, proposedPath):
#     dist = 0
#     i = 0
#     len_proposedPath = proposedPath.shape[0]
#     while i != len_proposedPath-1:
#         dist = dist + distMatrix.iloc[proposedPath[i], proposedPath[i+1]]
#         i = i + 1
#     totaldist = dist +distMatrix.iloc[proposedPath[-1], proposedPath[0]]
#     return totaldist


def distCalculator(distMatrix, proposedPath):
    dist = 0
    i = 0
    len_proposedPath = proposedPath.shape[0]
    for i in range(len_proposedPath-1):
        dist = dist + distMatrix.iloc[proposedPath[i], proposedPath[i+1]]
#     totaldist = dist +distMatrix.iloc[proposedPath[-1], proposedPath[0]] #don't need this as the list is already includes the depot
    return dist

In [14]:
#INPUTS: 
#D    - some given distance matrix (symmetric and 0's on diag)
#n    - the number of cities to vist

#OUTPUTS:
#path - the optimal path according to the simulated annealing
#dist - the corresponding distance to the optimal path
#distCal - the distance of the path calculated by the distance matrix
def simulated_annealing(D): 
    number_states = D.shape[0]
    n = number_states

    T = 100000 #Initial temperature
    L = 20*n #Length of the markov chain
    e = (1*exp(1))**(-9) #Tolerance
    
    #NOTE: x[0]= depot, x[1:n]=states, x[n+1]=depot    
    x0 = np.array([0])
    x_delivery_states  = np.random.permutation(range(1,number_states)) # creates an array of all the states including the depot in a random order. Note: num_states 
    x = np.concatenate((x0,x_delivery_states,x0), axis=0)
    fx = distCalculator(D, x) #calculates the distance of the random path
    
    while T > e:
        for i in range(0,L):
            rand1 = randint(1, n-1) #random integer between 1 and n-1, as includes n-1
            rand2 = randint(1, n-1)#random integer between 1 and n-1
            num1 = rand1#rand1*n
            num2 = rand2#rand2*n
            
            #ensuring num1 and num2 aren't equal
            while num1==num2:
                rand3 = randint(1, n-1)#random integer between 1 and n-1
                num1 = rand3#rand3*n
                
            y = x # copy of the random path and making random changes to compare two paths
            #make some changes to y_path
            #Note: changes can only be made to the delivery states
            swap1 = y[num1]
            y[num1] = y[num2]
            y[num2] = swap1
            fy = distCalculator(D,y)#calculate the distance of the y_path i.e the changes x_path
            
            if fy < fx:
                x = y
                fx = fy
            elif decimal.Decimal(random.randrange(100))/100 < exp(1)**(-(fy-fx)/T):
                x = y
                fx = fy
            
        T=0.9*T
    path = x
    dist = fx 
    dist_calc_path = distCalculator(D,x)
#     path_y = y
#     dist_y = fy 
#     dist_calc_path_y = distCalculator(D,y)

    return path, dist, dist_calc_path #path_y, dist_y, dist_calc_path_y

WHY IS THE OPTIMAL DISTANCE STILL THE SAME??
AS THE DEPOT AT THE START AND END ARE INCLUDED??

In [13]:
path, dist, distCalc = simulated_annealing(df_distMat)
display(path)
display(dist)
display(distCalc)

array([0, 2, 3, 4, 1, 5, 0])

229.0

279.0